In [1]:
import pandas as pd
import numpy as np
import re

## Section 1: Exploratory Data Analysis
This section of the notebook encompasses loading the training dataset and visualizing the summary statistics around the dataset. An understanding of the training and testing shape and distribution helps us understand key worksteps that we would need to take before beginning feature engineering and pre processing.  

In [2]:
train_df = pd.read_excel("C:/Users/abhij/Git Repos/WellsFargoChallenge-TransactionAnalytics/train.xlsx")
test_df = pd.read_excel("C:/Users/abhij/Git Repos/WellsFargoChallenge-TransactionAnalytics/test.xlsx")
train_df.head(5)

,sor,cdf_seq_no,trans_desc,merchant_cat_code,amt,db_cr_cd,payment_reporting_category,payment_category,is_international,default_brand,default_location,qrated_brand,coalesced_brand,Category
0,HH,T20110701260061756,RECUR DEBIT CRD PMT11/11 DELTA DENTAL OF A11 O...,6300.0,58.34,D,Card,Debit Card,False,DELTA DENTAL OF A11 OF,111-1111111 AR,Delta Dental,Delta Dental,Finance
1,HH,T201302289909010349,"CHECK CRD PURCHASE 11/11 SURETY SOLUTIONS, ...",NaN,103.00,D,Card,Check Card,False,"SURETY SOLUTIONS,",111-111-1111 OR,Surety Solutions,Surety Solutions,Finance
2,HH,T20130726991361190114550,CHECK CRD PURCHASE 11/11 THE COPY STOP ...,NaN,22.44,D,Card,Check Card,False,THE COPY STOP,SALT LAKE CIT UT,The Copy Stop,The Copy Stop,Finance
3,BK,T201207095780929968,MARKET ALERT INC 111-111-1111 TX,7375.0,22.44,NaN,Card,Credit Card,False,MARKET ALERT INC,111-111-1111 TX,Market Alert,Market Alert,Finance
4,HH,T20131230990558080004939,CHECK CRD PURCHASE 11/11 PERT- NER PERFECT ...,NaN,66.25,D,Card,Check Card,False,PERT- NER PERFECT,THE WOODLANDS TX,Hometown Insurance Partners,Hometown Insurance Partners,Finance


In [3]:
test_df.head(5)

,sor,cdf_seq_no,trans_desc,merchant_cat_code,amt,db_cr_cd,payment_reporting_category,payment_category,is_international,default_brand,default_location,qrated_brand,coalesced_brand,Category
0,HH,T20131230990668080055738,CHECK CRD PURCHASE 11/11 PACKAGE EXPRESS ...,NaN,10.35,D,Card,Check Card,False,PACKAGE EXPRESS,LACEY WA,Package Express,Package Express,NaN
1,HH,T201302289918775816,RECUR DEBIT CRD PMT11/11 YP *FRMLY AT&T AD ...,NaN,36.00,D,Card,Debit Card,False,YP *FRMLY AT&T AD,111-111-1111 CA,At And T,At And T,NaN
2,HH,T20130726991361190218055,CHECK CRD PURCHASE 11/11 NORMAN G JENSEN IN ...,NaN,27.00,D,Card,Check Card,False,NORMAN G JENSEN IN,111-1111111 MN,Norman G Jensen,Norman G Jensen,NaN
3,HH,T201208319924922772,CHECK CRD PUR RTRN 11/11 TWILIO ...,NaN,20.00,C,Card,Check Card,False,TWILIO,SAN FARANSICO CA,Twilio,Twilio,NaN
4,HH,T20131230990638080027066,CHECK CRD PURCHASE 11/11 AT&T D11K 1111 ...,NaN,325.78,D,Card,Check Card,False,AT&T D11K 1111,FORT WORTH TX,At And T,At And T,NaN


In [4]:
test_df.shape

(10000, 14)

An important thing to note is that the target feature is present in the test dataset as an empty column

In [5]:
train_df.describe()

,merchant_cat_code,amt
count,24691.000000,40000.000000
mean,6263.878134,75.870519
std,1217.403683,254.587351
min,0.000000,0.010000
25%,5699.000000,14.007500
50%,5814.000000,29.990000
75%,7230.000000,63.650000
max,9405.000000,23313.800000


In [6]:
test_df.describe()

,merchant_cat_code,amt,Category
count,6287.000000,10000.000000,0.0
mean,6281.211707,76.203879,NaN
std,1239.748198,269.585852,NaN
min,742.000000,0.010000,NaN
25%,5699.000000,14.000000,NaN
50%,5814.000000,29.955000,NaN
75%,7298.000000,62.602500,NaN
max,9399.000000,12537.330000,NaN


The distribution of amounts seem to be similar overall for train and test datasets. Below are the features and the type of features for the modelling problem:


In [7]:
print(train_df.dtypes)

sor                            object
cdf_seq_no                     object
trans_desc                     object
merchant_cat_code             float64
amt                           float64
db_cr_cd                       object
payment_reporting_category     object
payment_category               object
is_international                 bool
default_brand                  object
default_location               object
qrated_brand                   object
coalesced_brand                object
Category                       object
dtype: object


In [8]:
train_df['db_cr_cd'].value_counts()

D    37962
C     1726
Name: db_cr_cd, dtype: int64

In [9]:
# Creating a function to view target distribution by a column
def tgt_dist(df,target,col_name):
    df = pd.crosstab(index=df[col_name], 
                             columns=train_df[target],
                             margins=True).sort_values(by = 'All', ascending=False).head(20)

    return df
    

In [10]:
train_df.columns  

Index(['sor', 'cdf_seq_no', 'trans_desc', 'merchant_cat_code', 'amt',
       'db_cr_cd', 'payment_reporting_category', 'payment_category',
       'is_international', 'default_brand', 'default_location', 'qrated_brand',
       'coalesced_brand', 'Category'],
      dtype='object')

In [11]:
cross = pd.crosstab(index=train_df["default_brand"], 
                             columns=train_df["Category"],
                             margins=True).sort_values(by = 'All', ascending=False).head(20)

cross

Category,Communication Services,Education,Entertainment,Finance,Health and Community Services,Property and Business Services,Retail Trade,Services to Transport,"Trade, Professional and Personal Services",Travel,All
default_brand,,,,,,,,,,,
All,282,445,11255,185,4157,1095,13500,2317,5275,1489,40000
CVS PHARMACY #1111,0,0,0,0,0,0,3,0,0,0,3
WALGREENS #1111,0,0,0,0,0,0,3,0,0,0,3
LIVINGSOCIAL*,0,0,0,0,0,0,3,0,0,0,3
American Ski Exchange,0,0,0,0,0,0,2,0,0,0,2
MSL MAIN CENTRAL A,0,0,0,0,0,0,2,0,0,0,2
MOXIES CLASSIC GRILL #,0,0,2,0,0,0,0,0,0,0,2
MOO INC PRINTING,0,0,0,0,0,2,0,0,0,0,2
MONTE CARLO FRONT DESK,0,0,0,0,0,0,0,0,0,2,2


#### Important Features by intuition 
* Payment Category
* db_cr_cd
* default_brand

In [12]:

sample_desc = train_df.head(100).iloc[60,2]
sample_desc

'CHECK CRD PURCHASE 11/11 COUNTRY OAKS RV MOBILE  111-111-1111  LA 111111XXXXXX1111 111111111111111                               ?MCC=1111 11'

Many of the transaction descriptions have ones and random information that do not seem of relevance. Removing them before further steps of extracting features seem to be required before further modelling. Lets define a function that removes these unrequired tokens.    

## Data Cleaning and Preprocessing

This section of the notebook focuses on removing unrelated text to the problem statement. All transaction descriptions are then converted to lower text. to avoid redundancies during tokenization and word extraction. Finally, each transaction category which is the target variable is converted to a numerical value between 0 to 9.  

1. We first remove unwanted text(various combinations of 1s) found in transaction descriptions and default location.

In [13]:
def oneremover(df):
    desc_series = list(df['trans_desc'])
    for item in range(len(desc_series)):
        desc_series[item] = re.sub(" +", " ", desc_series[item])
        desc_series[item] = re.sub("111111XXXXXX1111 111111111111111", "", desc_series[item])
        desc_series[item] = re.sub("\?MCC=1111 11", "", desc_series[item])
        desc_series[item] = re.sub("111-111-1111", " ", desc_series[item])
        desc_series[item] = re.sub("111-1111111", "", desc_series[item])
        desc_series[item] = re.sub("11111", "", desc_series[item])
        desc_series[item] = re.sub("\?MCC=1111", "", desc_series[item])
        
    # Cleaning default location    
    desc_series_location = list(df['default_location'])
    desc_series_location = [str(p) for p in desc_series_location]
    for item in range(len(desc_series_location)):
        desc_series_location[item] = re.sub("111-111-1111", "", desc_series[item])
        desc_series_location[item] = re.sub("111-1111111", "", desc_series[item])
        desc_series_location[item] = re.sub("11111", "", desc_series[item])
        
    
        
    for index, row in df.iterrows():
        df.at[index, 'trans_desc'] = desc_series[index]
        df.at[index, 'default_location'] = desc_series[index]
    return df

In [14]:
train_df = oneremover(train_df)
train_df

,sor,cdf_seq_no,trans_desc,merchant_cat_code,amt,db_cr_cd,payment_reporting_category,payment_category,is_international,default_brand,default_location,qrated_brand,coalesced_brand,Category
0,HH,T20110701260061756,RECUR DEBIT CRD PMT11/11 DELTA DENTAL OF A11 O...,6300.0,58.34,D,Card,Debit Card,False,DELTA DENTAL OF A11 OF,RECUR DEBIT CRD PMT11/11 DELTA DENTAL OF A11 O...,Delta Dental,Delta Dental,Finance
1,HH,T201302289909010349,"CHECK CRD PURCHASE 11/11 SURETY SOLUTIONS, OR",NaN,103.00,D,Card,Check Card,False,"SURETY SOLUTIONS,","CHECK CRD PURCHASE 11/11 SURETY SOLUTIONS, OR",Surety Solutions,Surety Solutions,Finance
2,HH,T20130726991361190114550,CHECK CRD PURCHASE 11/11 THE COPY STOP SALT LA...,NaN,22.44,D,Card,Check Card,False,THE COPY STOP,CHECK CRD PURCHASE 11/11 THE COPY STOP SALT LA...,The Copy Stop,The Copy Stop,Finance
3,BK,T201207095780929968,MARKET ALERT INC TX,7375.0,22.44,NaN,Card,Credit Card,False,MARKET ALERT INC,MARKET ALERT INC TX,Market Alert,Market Alert,Finance
4,HH,T20131230990558080004939,CHECK CRD PURCHASE 11/11 PERT- NER PERFECT THE...,NaN,66.25,D,Card,Check Card,False,PERT- NER PERFECT,CHECK CRD PURCHASE 11/11 PERT- NER PERFECT THE...,Hometown Insurance Partners,Hometown Insurance Partners,Finance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,HH,T20130726991321130015959,CHECK CRD PURCHASE 11/11 ALLIED FIDELITY SE ...,NaN,100.00,D,Card,Check Card,False,ALLIED FIDELITY SE,CHECK CRD PURCHASE 11/11 ALLIED FIDELITY SE ...,Fidelity Investments,Fidelity Investments,Finance
39996,HH,T201208319925685298,CHECK CRD PURCHASE 11/11 FOREMOST INSURANCE MI,NaN,184.78,D,Card,Check Card,False,FOREMOST INSURANCE,CHECK CRD PURCHASE 11/11 FOREMOST INSURANCE MI,Foremost Insurance,Foremost Insurance,Finance
39997,HH,T20111004560028307,"CHECK CRD PURCHASE 11/11 PROPAY USA- ProPay, I...",7399.0,9.95,D,Card,Check Card,False,"PROPAY USA- ProPay, In","CHECK CRD PURCHASE 11/11 PROPAY USA- ProPay, I...",Propay,Propay,Finance
39998,HH,T20110601320040304,CHECK CRD PURCHASE 11/11 COWBOY FINANCE LLC NM,6051.0,250.00,D,Card,Check Card,False,COWBOY FINANCE LLC,CHECK CRD PURCHASE 11/11 COWBOY FINANCE LLC NM,Cowboy Finance,Cowboy Finance,Finance


In [18]:
train_df['trans_desc']

0        RECUR DEBIT CRD PMT11/11 DELTA DENTAL OF A11 O...
1        CHECK CRD PURCHASE 11/11 SURETY SOLUTIONS,   OR  
2        CHECK CRD PURCHASE 11/11 THE COPY STOP SALT LA...
3                                    MARKET ALERT INC   TX
4        CHECK CRD PURCHASE 11/11 PERT- NER PERFECT THE...
                               ...                        
39995    CHECK CRD PURCHASE 11/11 ALLIED FIDELITY SE   ...
39996    CHECK CRD PURCHASE 11/11 FOREMOST INSURANCE  MI  
39997    CHECK CRD PURCHASE 11/11 PROPAY USA- ProPay, I...
39998    CHECK CRD PURCHASE 11/11 COWBOY FINANCE LLC  NM  
39999    CHECK CRD PURCHASE 11/11 JONJON DELI GROCER BR...
Name: trans_desc, Length: 40000, dtype: object

## Baseline Modelling 
** without any Feature Engineering or Data Cleaning


In [15]:
## Tokenization
